# BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Query$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $Doc$ запросу $Query$:

$$ BM25(Query, Doc) = \sum_{i=1}^{n} \text{IDF}(q_i)*\frac{TF(q_i,Doc)*(k+1)}{TF(q_i,Doc)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где    
$$$$
$\text{IDF}(q_i)$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в корпусе   
$n(q_i)$ — количество документов, содержащих слово $q_i$

>$TF(q_i,Doc)$ - частота слова $q_i$ в документе $Doc$    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75  
$l(d)$ - количество слов документа  
$avgdl$ — средняя длина документов в корпусе    

## Распишем операции

Первая компонента **IDF**. Переменные для расчета:
- $N$ - общее число документов в корпусе (*shape*)
- $n(q_i)$ - количество документов, содержащих слово $q_i$ (*numpy.count_nonzero*)

Вторая компонента **TF * Const**. Переменные для расчета:
- $TF(q_i,Doc)$ - частота слова $q_i$ в документе $Doc$ (*CountVectorizer*) 
- $l(d)$ - количество слов в документе (*DT_CountVectorizer_Matrix.sum(axis=1)*)
- $avgdl$ — среднее количество слов документа в корпусе (*DT_CountVectorizer_Matrix.sum(axis=1).mean()*) 
- Константы $k$=2.0 и $b$=0.75  

# run in dense


$$ BM25(Query, Doc) = \sum_{i=1}^{n} \text{IDF}(q_i)*\frac{TF(q_i,Doc)*(k+1)}{TF(q_i,Doc)+k(1-b+b\frac{l(d)}{avgdl})} $$ 

Начинаем реализацию BM25

In [2]:
# импорты, константы и корпус 

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 

k = 2
b = 0.75

texts = [
    'киса',
    'мама',
    'мыла',
    'раму',
    'киса-мама мыла раму'
]

# from sklearn.datasets import fetch_20newsgroups
# texts = fetch_20newsgroups(subset='train')['data']
# texts = texts[:10000]

Создадим основные переменные - TF и IDF

In [3]:
# матрица tf + понадобится для индексации запроса
count_vectorizer = CountVectorizer()
count = count_vectorizer.fit_transform(texts).toarray() 
tf = count

# для расчета idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True, norm='l2') 
tfidf = tfidf_vectorizer.fit_transform(texts).toarray()

# расчет idf
idf = tfidf_vectorizer.idf_  # формула idf в sklearn: log((N+1)/(n+1))+1, и нам это ок
idf = np.expand_dims(idf, axis=0) # необязательно благодаря broadcast 

Создадим переменные знаменателя - l(d) и avdl

In [4]:
# расчет количества слов в каждом документе - l(d) 
len_d = tf.sum(axis=1)

# расчет среднего количества слов документов корпуса - avdl
avdl = len_d.mean()

Создадим итоговую матрицу для корпуса

In [5]:
# расчет числителя
A = idf * tf * (k + 1)

# расчет знаменателя
B_1 = (k * (1 - b + b * len_d / avdl))
B_1 = np.expand_dims(B_1, axis=-1) 

B = tf + B_1

# BM25
matrix = A / B

Посчитаем релевантность документов для запроса

In [6]:
query = 'киса'
# преобразуем запрос в вектор 
query_count_vec = count_vectorizer.transform([query]).toarray()

In [7]:
%%time
np.dot(matrix, query_count_vec.T)

CPU times: user 42 µs, sys: 19 µs, total: 61 µs
Wall time: 66.8 µs


array([[2.08387345],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.96751267]])

# run in sparse

Способ создания разреженной матрицы

In [8]:
from scipy import sparse

data = np.array([18, 26, 3, 4, 54, 6])
row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])

sparse.csr_matrix((data, (row, col)), shape=(3, 3)).toarray()

array([[18,  0, 26],
       [ 0,  0,  3],
       [ 4, 54,  6]], dtype=int64)